In [1]:
import mapd
import torchvision

In [2]:
MNIST_ROOT = "data"
torchvision.datasets.MNIST(root=MNIST_ROOT, download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train

In [3]:
from torchvision.datasets import MNIST

In [4]:
from torch.utils.data import Dataset

class IDXDataset(Dataset):
    def __init__(self, dataset: Dataset):
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return self.dataset[index], index

In [5]:
from abc import ABCMeta, abstractmethod

import torch
from lightning import LightningModule
from typing import Any, Optional, List, Dict
from torch import Tensor
from torch.utils.data import DataLoader


class MAPDModule(LightningModule, metaclass=ABCMeta):
    mapd_current_indices_: Optional[torch.Tensor]
    mapd_losses_: Tensor
    mapd_indices_: Tensor

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.mapd_current_indices_ = []
        self.mapd_losses_ = torch.empty(0, dtype=torch.float32)
        self.mapd_indices_ = torch.empty(0, dtype=torch.int64)

    @classmethod
    @abstractmethod
    def batch_loss(self, logits: Any, y: Any) -> Tensor:
        raise NotImplemented("batch_loss method not implemented")

    def on_before_batch_transfer(self, batch: Any, dataloader_idx: int) -> Any:
        batch, indices = batch
        self.mapd_current_indices_ = indices
        return batch

    def mapd_log(self, logits: Any, y: Any):
        loss = self.batch_loss(logits, y).detach()
        current_indices = self.mapd_current_indices_

        self.mapd_losses_ = torch.cat((self.mapd_losses_, loss))
        self.mapd_indices_ = torch.cat((self.mapd_indices_, current_indices))

    def _reset_mapd_attrs(self):
        self.mapd_losses_ = torch.empty(0, dtype=torch.float32)
        self.mapd_indices_ = torch.empty(0, dtype=torch.int64)

    def on_train_epoch_end(self) -> None:
        #self._reset_mapd_attrs()
        pass

    def on_validation_epoch_end(self):
        # Run loss logging for probes
        #dataloader = self.probe_suite_dataloader()
        pass

In [6]:
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x
    
model = Net()

In [7]:
import lightning as L
from torch.nn import functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch


class ResNet18(MAPDModule):
    def __init__(
        self,
        max_epochs: int = 10,
        lr: float = 0.05,
        momentum: float = 0.9,
        weight_decay: float = 0.0005
    ):
        super().__init__()
        self.model = model

        self.max_epochs = max_epochs
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay

        self.save_hyperparameters(ignore=["model"])

    def forward(self, x):
        return self.model(x)
    
    def batch_loss(self, logits, y) -> torch.Tensor:
        return F.cross_entropy(logits, y, reduction="none")

    def training_step(self, batch, batch_idx):
        x, y = batch
        
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        self.mapd_log(logits, y)
        
        return loss

    def configure_optimizers(self):
        optimizer = SGD(
            self.parameters(),
            lr=self.lr
        )

        return {"optimizer": optimizer}

In [8]:
from torch.utils.data import random_split, DataLoader
from torchvision import transforms

class MNISTDataModule(L.LightningDataModule):
    def __init__(self, data_dir: str = "data", batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    def setup(self, stage: str):
        self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)
        self.mnist_predict = MNIST(self.data_dir, train=False, transform=self.transform)
        mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        self.mnist_train = IDXDataset(self.mnist_train)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.mnist_predict, batch_size=self.batch_size)

In [9]:
module = ResNet18()
dm = MNISTDataModule()

trainer = L.Trainer(accelerator="cpu", max_epochs=1)

trainer.fit(module, datamodule=dm)

/home/jonashoffmann/miniconda3/envs/mapd3/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jonashoffmann/miniconda3/envs/mapd3/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/home/jonashoffmann/miniconda3/envs/mapd3/lib/python3.9/site-packages/lightning/pytorch/tr

Training: 0it [00:00, ?it/s]

/home/jonashoffmann/miniconda3/envs/mapd3/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:370: UserWarning: You have overridden `on_before_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=1` reached.
